<a href="https://colab.research.google.com/github/omarelsebaey002/test01/blob/main/automated_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to run**


1.   Upload helper.py to a folder on the drive
      *   For example create a folder called "trading app" and then upload the helper.py to this folder


2. Set the path to the helper.py in the next cell in this line 
      * sys.path.append('/content/drive/MyDrive/trading_app')

3. For single run 
      * Set the variables as elaborated in the cell#2

4. For multi runs with a file containing the input files 
      * Call the funciton generate_net_threshold_for_multi_runs and pass the file to it like Cell#3
      * The columns of the file should have columns names like this : 
        * input_file,output_file,ticker,start_date,end_date


In [4]:
import os
from google.colab import drive
import sys
sys.path.append('/content/drive/MyDrive/trading_app')
from helper import *
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Single Run example

In [5]:
outputfile         = '/content/drive/MyDrive/trading_app/net_thr.csv'
input_file         = '/content/drive/MyDrive/trading_app/small.csv'
START_DATE         = '06-08-21 9:15'
END_DATE           = '06-08-21 9:50'
TICKER             = ""
generate_net_threshold(input_file,START_DATE,END_DATE,TICKER,outputfile)

[INFO] Processing input file: /content/drive/MyDrive/trading_app/small.csv
[INFO] Selecting from START_DATE:  2021-08-06 09:15:00
[INFO]           to   START_DATE:  2021-08-06 09:50:00
[INFO] Found 30 entries matching the selection criteria 
[INFO] Processing Start...
[INFO] Writing the output to: /content/drive/MyDrive/trading_app/net_thr.csv 
[INFO] FINISHED!


Multiple Runs from a csv file 

In [6]:
multi_files_input = '/content/drive/MyDrive/trading_app/input_file.csv'
generate_net_threshold_for_multi_runs(multi_files_input)

[INFO] Generating for file number:  0
[INFO] Processing input file: /content/drive/MyDrive/trading_app/small.csv
[INFO] Ticker is set to : BANKNIFTY and Swing will be +/- 300
[INFO] Selecting from START_DATE:  2021-08-06 09:15:00
[INFO]           to   START_DATE:  2021-08-06 09:31:00
[INFO] Found 17 entries matching the selection criteria 
[INFO] Processing Start...
[INFO] Writing the output to: /content/drive/MyDrive/trading_app/net_thr.csv 
[INFO] FINISHED!
[INFO] Generating for file number:  1
[INFO] Processing input file: /content/drive/MyDrive/trading_app/small.csv
[INFO] Selecting from START_DATE:  2021-08-06 09:15:00
[INFO]           to   START_DATE:  2021-08-06 09:31:00
[INFO] Found 17 entries matching the selection criteria 
[INFO] Processing Start...
[INFO] Writing the output to: /content/drive/MyDrive/trading_app/net_thr.csv 
[INFO] FINISHED!
[INFO] Generating for file number:  2
[INFO] Processing input file: /content/drive/MyDrive/trading_app/small.csv
[INFO] Ticker is set 